<a href="https://colab.research.google.com/github/RyuichiSaito1/covid19-twitter-usa-restoring/blob/main/convert_tsv_to_jsonl_for_gpt3_5_finetuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

!pip install --upgrade openai

!pip install pandas

In [ ]:
import json
import pandas as pd

# Path to the TSV file on Google Drive
tsv_file_path = '/content/drive/MyDrive/covid-twitter-usa-normal/data/training_data/gpt-3.5/validation_data_2021_shuffle_majority_vote_gpt3.5.tsv'

# Path to save the JSONL file
jsonl_file_path = '/content/drive/MyDrive/covid-twitter-usa-normal/data/training_data/gpt-3.5/validation_data_2021_shuffle_majority_vote_gpt3.5.jsonl'

def convert_to_new_format(prompt, completion):
    # Convert the data to the new JSONL format
    new_data = {
        "messages": [
            {"role": "system", "content": "You are a sentiment classifier of tweets from citizens of a large US city collected during the COVID-19 pandemic. Classify into three values: 0 for positive, 1 for neutral, and 2 for negative. Positive sentiment include: admire, amazing, assure, celebration, charm, eager, enthusiastic, excellent, fancy, fantastic, frolic, graceful, happy, joy, luck, majesty, mercy, nice, patience, perfect, proud, rejoice, relief, respect, satisfactorily, sensational, super, terrific, thank, vivid, wise, wonderful, zest, expectations and etc. Negative sentiment include: abominable, anger, anxious, bad, catastrophe, cheap, complaint, condescending, deceit, defective, disappointment, embarrass, fake, fear, filthy, fool, guilt, hate, idiot, inflict, lazy, miserable, mourn, nervous, objection, pest, plot, reject, scream, silly, terrible, unfriendly, vile, wicked, and etc. Neutral sentiment: neither positive or negative, such as text without sentiment, stating a fact, question, news article, advertisement, solicitation, request, quote, unintelligible text, and etc. When the sentiment is mixed, such as both joy and sadness, use your judgment to choose the stronger emotion."},
            {"role": "user", "content": prompt},
            {"role": "assistant", "content": str(completion)} # Surround completion with double quotes
        ]
    }
    return new_data

# Read the TSV file using pd.read_table, skipping the first row as header
df = pd.read_table(tsv_file_path, names=('Prompt', 'Completion'), skiprows=1)

# Convert to JSONL format and save
top_10_data = []
with open(jsonl_file_path, 'w') as jsonl_file:
    for index, row in df.iterrows():
        prompt_text = row['Prompt']
        completion_text = row['Completion']

        new_data = convert_to_new_format(prompt_text, completion_text)

        # Write each data as a line in the JSONL file with ensure_ascii=False
        jsonl_file.write(json.dumps(new_data, ensure_ascii=False) + '\n')

        # Collect data for top 10 output
        top_10_data.append((prompt_text, completion_text))

print(f"Conversion completed. JSONL file saved at: {jsonl_file_path}")

# Display the top 10 data
print("Top 10 Data:")
for i, (prompt, completion) in enumerate(top_10_data[:10]):
    print(f"{i + 1}. Prompt: {prompt}\n   Completion: {completion}\n")